# Question Answering with LangChain, OpenAI, and MultiQuery Retriever

This interactive workbook demonstrates example of Elasticsearch's [MultiQuery Retriever](https://api.python.langchain.com/en/latest/retrievers/langchain.retrievers.multi_query.MultiQueryRetriever.html) to generate similar queries for a given user input and apply all queries to retrieve a larger set of relevant documents from a vectorstore.

Before we begin, we first split the fictional workplace documents into passages with `langchain` and uses OpenAI to transform these passages into embeddings and then store these into Elasticsearch.

We will then ask a question, generate similar questions using langchain and OpenAI, retrieve relevant passages from the vector store, and use langchain and OpenAI again to provide a summary for the questions.

## Install packages and import modules

In [1]:
!pip install -U langchain langchain-community langchain-openai

In [2]:
!python3 -m pip install -qU jq lark langchain langchain-elasticsearch langchain_openai tiktoken

from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_elasticsearch import ElasticsearchStore
from langchain_openai.llms import OpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
from getpass import getpass



## Connect to Elasticsearch

ℹ️ We're using an Elastic Cloud deployment of Elasticsearch for this notebook. If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?utm_source=github&utm_content=elasticsearch-labs-notebook) for a free trial.

We'll use the **Cloud ID** to identify our deployment, because we are using Elastic Cloud deployment. To find the Cloud ID for your deployment, go to https://cloud.elastic.co/deployments and select your deployment.

We will use [ElasticsearchStore](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.elasticsearch.ElasticsearchStore.html) to connect to our elastic cloud deployment, This would help create and index data easily.  We would also send list of documents that we created in the previous step

In [4]:
from getpass import getpass
from langchain_community.vectorstores.elasticsearch import ElasticsearchStore
from langchain_openai import OpenAIEmbeddings

# Prompt securely for credentials
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")
ELASTIC_API_KEY = getpass("Elastic API Key: ")
OPENAI_API_KEY = getpass("OpenAI API Key: ")

# Create OpenAI embedding model
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# Set a meaningful index name
INDEX_NAME = "workplace-docs"  # or any name you prefer

# Create Elasticsearch vector store
vectorstore = ElasticsearchStore(
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
    index_name=INDEX_NAME,
)


Elastic Cloud ID: ··········
Elastic API Key: ··········
OpenAI API Key: ··········


## Indexing Data into Elasticsearch
Let's download the sample dataset and deserialize the document.

In [5]:
from urllib.request import urlopen
import json

url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/example-apps/chatbot-rag-app/data/data.json"

response = urlopen(url)
data = json.load(response)

with open("temp.json", "w") as json_file:
    json.dump(data, json_file)

### Split Documents into Passages

We’ll chunk documents into passages in order to improve the retrieval specificity and to ensure that we can provide multiple passages within the context window of the final question answering prompt.

Here we are chunking documents into 800 token passages with an overlap of 400 tokens.

Here we are using a simple splitter but Langchain offers more advanced splitters to reduce the chance of context being lost.

### Bulk Import Passages

Now that we have split each document into the chunk size of 800, we will now index data to elasticsearch using [ElasticsearchStore.from_documents](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.elasticsearch.ElasticsearchStore.html#langchain.vectorstores.elasticsearch.ElasticsearchStore.from_documents).

We will use Cloud ID, Password and Index name values set in the `Create cloud deployment` step.

In [9]:
from dateutil.parser import parse

def metadata_func(record: dict, metadata: dict) -> dict:
    metadata['name'] = record.get('name', 'unknown-source')
    metadata['summary'] = record.get('summary', '')
    metadata['url'] = record.get('url', '')
    metadata['category'] = record.get('category', '')

    updated_at = record.get('updated_at')
    if updated_at and updated_at.strip():
        try:
            dt = parse(updated_at)
            metadata['updated_at'] = dt.isoformat()
        except Exception as e:
            print(f"Warning: Failed to parse date '{updated_at}' - {e}")
            metadata.pop('updated_at', None)
    else:
        metadata.pop('updated_at', None)

    print(f"Processing record, updated_at: {updated_at} -> metadata: {metadata.get('updated_at', 'Not set')}")
    return metadata


In [12]:
from langchain.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

loader = JSONLoader(
    file_path="temp.json",
    jq_schema=".[]",
    content_key="content",
    metadata_func=metadata_func,
)
raw_docs = loader.load()
for i, doc in enumerate(raw_docs[:5]):
    print(f"Raw Doc {i+1} content:", doc.page_content)
    print(f"Raw Doc {i+1} metadata:", doc.metadata)




docs = loader.load_and_split(text_splitter=text_splitter)

for i, doc in enumerate(docs[:3]):
    print(f"Doc {i+1} metadata:", doc.metadata)

for doc in docs[:3]:
    print(doc.metadata)  # Should include 'name'


for doc in docs:
    if 'name' not in doc.metadata or not doc.metadata['name']:
        doc.metadata['name'] = "unknown-source"

vectorstore = ElasticsearchStore.from_documents(
    docs,                # these docs with metadata!
    embeddings,
    index_name="search-ipd3",
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
)



Processing record, updated_at: 2020-03-01 -> metadata: 2020-03-01T00:00:00
Processing record, updated_at: 2022-04-29 -> metadata: 2022-04-29T00:00:00
Processing record, updated_at: 2023-05-01 -> metadata: 2023-05-01T00:00:00
Processing record, updated_at: 2023-04-15 -> metadata: 2023-04-15T00:00:00
Processing record, updated_at: 2018-04-16 -> metadata: 2018-04-16T00:00:00
Processing record, updated_at: None -> metadata: Not set
Processing record, updated_at: None -> metadata: Not set
Processing record, updated_at: None -> metadata: Not set
Processing record, updated_at: None -> metadata: Not set
Processing record, updated_at: None -> metadata: Not set
Processing record, updated_at: None -> metadata: Not set
Processing record, updated_at: None -> metadata: Not set
Processing record, updated_at: None -> metadata: Not set
Processing record, updated_at: None -> metadata: Not set
Processing record, updated_at: None -> metadata: Not set
Raw Doc 1 content: Effective: March 2020
Purpose

The p

In [13]:
vectorstore = ElasticsearchStore.from_documents(
    docs,
    embeddings,
    index_name="search-ipd3",
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
)

llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

retriever = MultiQueryRetriever.from_llm(vectorstore.as_retriever(), llm)


# Question Answering with MultiQuery Retriever

Now that we have the passages stored in Elasticsearch, we can now ask a question to get the relevant passages.

In [17]:
from langchain.llms import OpenAI
from langchain.vectorstores.elasticsearch import ElasticsearchStore
from langchain.retrievers import MultiQueryRetriever
from langchain.schema.runnable import Runnable, RunnableParallel, RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import format_document
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

# Create vectorstore from documents (docs, embeddings, ELASTIC_* keys assumed defined)
vectorstore = ElasticsearchStore.from_documents(
    docs,
    embeddings,
    index_name="search-ipd3",
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
)

# Initialize OpenAI LLM
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

# Create MultiQueryRetriever from vectorstore retriever + LLM
retriever = MultiQueryRetriever.from_llm(vectorstore.as_retriever(), llm)

# Prompt templates
LLM_CONTEXT_PROMPT = ChatPromptTemplate.from_template(
    """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Be as verbose and educational in your response as possible.

    context: {context}
    Question: "{question}"
    Answer:
    """
)

LLM_DOCUMENT_PROMPT = PromptTemplate.from_template(
    """
---
SOURCE: {name}
{page_content}
---
"""
)

# Runnable class for combining docs into a single string
class CombineDocumentsRunnable(Runnable):
    def invoke(self, docs, *args, **kwargs):
        for doc in docs:
            if 'name' not in doc.metadata or not doc.metadata['name']:
                doc.metadata['name'] = "unknown-source"
        doc_strings = [format_document(doc, LLM_DOCUMENT_PROMPT) for doc in docs]
        return "\n\n".join(doc_strings)




_combine_documents = CombineDocumentsRunnable()

# Build the runnable chain
_context = RunnableParallel(
    context=retriever | _combine_documents,
    question=RunnablePassthrough(),
)

chain = _context | LLM_CONTEXT_PROMPT | llm

# Ask your question
ans = chain.invoke("what is the nasa sales team?")

print("---- Answer ----")
print(ans)


INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you provide information on the sales team at NASA?', '2. How does the sales team operate within NASA?', '3. What are the responsibilities of the NASA sales team?']


---- Answer ----
The NASA sales team is a part of the Americas region in the sales organization. It is responsible for serving customers in North America and South America, with two Area Vice-Presidents, Laura Martinez and Gary Johnson. The team works closely with other departments to identify and pursue new business opportunities, nurture existing client relationships, and ensure customer satisfaction.


**Generate at least two new iteratioins of the previous cells - Be creative.** Did you master Multi-
Query Retriever concepts through this lab?

In [18]:
answer = chain.invoke("What is the NASA sales team and how do they operate?")

print("---- Chain-of-Thought Answer ----")
print(answer)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. How does the NASA sales team function and what are their operations?', '2. Can you explain the operations of the NASA sales team and their role?', '3. What are the functions and operations of the NASA sales team?']


---- Chain-of-Thought Answer ----
The NASA sales team is a part of the Americas region in the sales organization. It is responsible for serving customers in North America and South America. The team is led by two Area Vice-Presidents, Laura Martinez for North America and Gary Johnson for South America. The team consists of dedicated account managers, sales representatives, and support staff who work together to identify and pursue new business opportunities, maintain existing client relationships, and ensure customer satisfaction. They also collaborate closely with other departments, such as marketing, product development, and customer support, to deliver high-quality products and services to clients.


In [20]:
answer = chain.invoke("How does NASA approach sustainability and environmental impact?")

print("---- Chain-of-Thought Answer ----")
print(answer)

INFO:langchain.retrievers.multi_query:Generated queries: ["1. What is NASA's approach to sustainability and environmental impact?", '2. How does NASA address sustainability and environmental impact?', '3. In what ways does NASA tackle sustainability and environmental impact?']


---- Chain-of-Thought Answer ----

NASA, or the North America South America region, is a part of the sales organization that serves customers in the United States, Canada, Mexico, Central America, and South America. While the main focus of NASA is on achieving business objectives and serving customers, it is also important to note that NASA is committed to sustainability and minimizing its environmental impact. This is evident in the fact that NASA has two Area Vice-Presidents, Laura Martinez and Gary Johnson, who oversee the region and are responsible for ensuring that sustainability practices are implemented and environmental impact is minimized. This approach to sustainability and environmental impact is crucial in today's world, where businesses are increasingly expected to prioritize and address environmental concerns. By having dedicated leaders and a structured organization, NASA is able to effectively incorporate sustainability into its operations and contribute to a more susta